In [1]:
from model import SirenImageLearner, SirenImageDataWrapper

import math
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import pytorch_lightning as pl

from torchvision.transforms import ToTensor, ToPILImage

from loguru import logger
import matplotlib.pyplot as plt
import PIL
import tqdm

# fuck it...
DEVICE = 'cuda'

In [2]:
!nvidia-smi

Tue Oct 12 05:30:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
|  0%   46C    P8     9W / 350W |    719MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
im_path = '../data/google-photos-export/uncompressed/takeout-20210901T023707Z-001/Takeout/Google Photos/lithophane candidates/20210313_133327.jpg'
im = PIL.Image.open(im_path)
im3 = im.resize((122, 163)).convert('L').rotate(-90, expand=True)
t_im3 = ToTensor()(im3).squeeze()

from pytorch_lightning.loggers import TensorBoardLogger, MLFlowLogger, WandbLogger

#wandb_logger = WandbLogger()
tb_logger = TensorBoardLogger("tb_logs", name="siren-upsample")

model = SirenImageLearner(
    #notebook_mode=True
    warmup_steps=150,
    )
dm = SirenImageDataWrapper(target_image=t_im3)

# https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html#trainer-flags
trainer = pl.Trainer(
    gpus=-1,
    max_steps=1000,
    #max_steps=100000, # I think there's a way to use CLI args to override here
    log_every_n_steps=100, #50,
    logger=tb_logger,
    log_gpu_memory=True, # fuck yeah!
    weights_summary=None,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [4]:
!nvidia-smi

Tue Oct 12 05:30:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
|  0%   46C    P8    11W / 350W |    719MiB / 24576MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
trainer.fit(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training: -1it [00:00, ?it/s]

In [6]:
!nvidia-smi

Tue Oct 12 05:31:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
|  0%   59C    P2   129W / 350W |   1849MiB / 24576MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# To do: experiment with regularization

# To do: WandB logging?

# To do: hparams parallel coords? wandb have this already? 
# Maybe easier with MLFlow logging to local db serving a custom dashboard

In [8]:
# Quantile approximation for distributional similarity

import numpy as np

t_im3.shape # 122, 163
t_rand = torch.rand(t_im3.shape)
n_quantiles = 100

# I like the idea of taking random quantiles like this, but I think it complicates
# the integral approximation. If quantiles are evenly spaced, I can just sum. 
# otherwise, I think I need to factor in the dx
##q = torch.rand(n_quantiles).sort().values
#q = torch.linspace(0,1,steps=n_quantiles)

#p0 = t_im3.quantile(q).numpy()
#p1 = t_rand.quantile(q).numpy()
#q = q.numpy()
#d = np.abs(p0-p1)

def q_q_distance(t0, t1, n_quantiles=100):
    q = torch.linspace(0,1,steps=n_quantiles+1)[:-1]
    p0 = t0.quantile(q)
    p1 = t1.quantile(q)
    d = math.sum(math.abs(p0-p1))
    return d

In [9]:
#plt.scatter(q, p0)
#plt.scatter(q, p1)
#plt.show()


#plt.scatter(q, d)
#plt.show()

In [10]:
#d.sum()

In [11]:
 # not identical
#p0[0], p1[0], p0[-1], p1[-1] # ends up being the max, basically identical

In [12]:
# https://discuss.pytorch.org/t/torch-equivalent-of-numpy-random-choice/16146/19
def perm_gpu_f32(pop_size, num_samples):
    """Use torch.randperm to generate indices on a 32-bit GPU tensor."""
    return torch.randperm(pop_size, dtype=torch.int32, device='cuda')[:num_samples]


In [13]:
perm_gpu_f32(10,20) # this is specifically w/o replacement

tensor([3, 1, 6, 2, 0, 8, 4, 7, 9, 5], device='cuda:0', dtype=torch.int32)